# Session 2: GenAI vs Agentic AI — Interactive Demos

**Audience:** Banking Technologists  
**Duration:** Run cells live during training (~60 min of demos)  
**Focus:** Architectural differences, autonomy levels, when to use each paradigm

---

## Setup — Run This First

In [ ]:
# Install required packages (run once)
!pip install -q anthropic tiktoken rich

In [ ]:
import os
import json
import time
from datetime import datetime
from typing import List, Dict, Any, Optional
from rich import print as rprint
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.markdown import Markdown

console = Console()

print("\n✅ All imports ready! Let's explore GenAI vs Agentic AI.\n")

---

## Demo 1: GenAI Stateless Architecture (Reactive)

**Concept:** GenAI has NO memory between calls. Each request is independent.

**Banking Context:** Customer service rep using GenAI for loan questions

In [ ]:
class GenAISimulator:
    """Simulates stateless GenAI behavior (no real API calls)"""
    
    def __init__(self):
        self.call_count = 0
        # GenAI has NO memory - each call is fresh
    
    def generate(self, prompt: str) -> str:
        """Each call is independent - no context from previous calls"""
        self.call_count += 1
        
        console.print(f"\n[cyan]═══ GenAI Call #{self.call_count} ═══[/cyan]")
        console.print(f"[yellow]Prompt:[/yellow] {prompt}")
        
        # Simulate GenAI responses
        if "credit score" in prompt.lower() and "720" in prompt:
            response = "A credit score of 720 is considered good. It indicates the borrower has a solid credit history with low default risk."
        elif "what was the credit score" in prompt.lower() and "720" not in prompt:
            # GenAI has NO memory!
            response = "I don't have information about a credit score in your question. Please provide the credit score you'd like me to analyze."
        elif "what was the credit score" in prompt.lower() and "720" in prompt:
            response = "The credit score mentioned is 720."
        elif "income" in prompt.lower() and "debt" in prompt.lower():
            response = "To calculate debt-to-income ratio (DTI), divide total monthly debt by gross monthly income, then multiply by 100. For example: ($1,200 / $7,083) × 100 = 16.9%"
        elif "customer name" in prompt.lower():
            response = "I don't have access to previous conversation context. Please provide the customer name you're asking about."
        else:
            response = "I'd be happy to help! Please provide more details about what you need."
        
        console.print(f"[green]Response:[/green] {response}\n")
        time.sleep(0.5)  # Simulate API latency
        return response


# Demonstrate stateless behavior
genai = GenAISimulator()

console.print("[bold magenta]DEMO: GenAI Stateless Architecture[/bold magenta]")
console.print("[dim]Watch how GenAI forgets context between calls...\n[/dim]")

# Call 1: Analyze credit score
genai.generate("The customer has a credit score of 720. Is this good?")

# Call 2: Try to reference previous context - FAILS!
genai.generate("What was the credit score we just discussed?")

# Call 3: Must repeat context
genai.generate("The credit score is 720. What was the credit score?")

console.print("\n[bold red]Key Insight:[/bold red] GenAI is STATELESS. You are the database!")
console.print("[dim]→ Human must manage context across calls[/dim]")
console.print("[dim]→ No memory of previous conversations[/dim]")
console.print("[dim]→ Each call is independent\n[/dim]")

---

## Demo 2: Agentic AI Stateful Architecture (Proactive)

**Concept:** Agents maintain memory and context across actions

**Banking Context:** Agent processing loan application autonomously

In [ ]:
class AgenticAISimulator:
    """Simulates stateful Agentic AI behavior"""
    
    def __init__(self):
        self.memory = {}  # Agent maintains state!
        self.execution_log = []
        self.step_count = 0
    
    def execute_goal(self, goal: str, context: Dict[str, Any]):
        """Agent autonomously plans and executes toward goal"""
        console.print(f"\n[bold cyan]═══ AGENTIC AI: Goal-Driven Execution ═══[/bold cyan]")
        console.print(f"[yellow]Goal:[/yellow] {goal}")
        console.print(f"[yellow]Initial Context:[/yellow] {json.dumps(context, indent=2)}\n")
        
        # Store context in memory
        self.memory.update(context)
        
        # AUTONOMOUS PLANNING
        console.print("[bold green]🧠 PLANNING PHASE[/bold green]")
        plan = [
            "Retrieve customer credit score",
            "Calculate debt-to-income ratio",
            "Check against lending policy",
            "Generate recommendation",
            "Create audit trail"
        ]
        for i, step in enumerate(plan, 1):
            console.print(f"  {i}. {step}")
        
        # AUTONOMOUS EXECUTION
        console.print("\n[bold green]⚙️  EXECUTION PHASE[/bold green]")
        
        # Step 1: Retrieve credit score (simulated tool call)
        self._execute_step(
            "Retrieve customer credit score",
            "query_credit_bureau",
            {"customer_id": self.memory['customer_id']}
        )
        self.memory['credit_score'] = 720  # Simulated result
        console.print(f"  ✓ Retrieved: Credit score = {self.memory['credit_score']}")
        
        # Step 2: Calculate DTI (agent remembers income from context!)
        self._execute_step(
            "Calculate debt-to-income ratio",
            "calculate_dti",
            {
                "monthly_income": self.memory['monthly_income'],
                "monthly_debt": self.memory['monthly_debt']
            }
        )
        dti = (self.memory['monthly_debt'] / self.memory['monthly_income']) * 100
        self.memory['dti_ratio'] = round(dti, 1)
        console.print(f"  ✓ Calculated: DTI = {self.memory['dti_ratio']}%")
        
        # Step 3: Check policy
        self._execute_step(
            "Check against lending policy",
            "check_policy",
            {"credit_score": self.memory['credit_score'], "dti": self.memory['dti_ratio']}
        )
        meets_policy = self.memory['credit_score'] >= 680 and self.memory['dti_ratio'] <= 43
        self.memory['meets_policy'] = meets_policy
        console.print(f"  ✓ Policy Check: {'PASS' if meets_policy else 'FAIL'}")
        
        # Step 4: Generate recommendation
        self._execute_step(
            "Generate recommendation",
            "generate_recommendation",
            {"customer": self.memory['customer_name'], "meets_policy": meets_policy}
        )
        recommendation = "APPROVE" if meets_policy else "DENY"
        self.memory['recommendation'] = recommendation
        console.print(f"  ✓ Recommendation: {recommendation}")
        
        # Step 5: Audit trail
        self._execute_step(
            "Create audit trail",
            "log_decision",
            {"decision": recommendation}
        )
        
        # RESULT
        console.print("\n[bold green]📊 RESULT[/bold green]")
        result = {
            "customer": self.memory['customer_name'],
            "credit_score": self.memory['credit_score'],
            "dti_ratio": self.memory['dti_ratio'],
            "recommendation": self.memory['recommendation'],
            "reasoning": f"Credit score {self.memory['credit_score']} (min 680), DTI {self.memory['dti_ratio']}% (max 43%)",
            "steps_executed": self.step_count
        }
        console.print(Panel(json.dumps(result, indent=2), title="Agent Result", border_style="green"))
        
        return result
    
    def _execute_step(self, description: str, tool: str, params: Dict):
        """Execute a single step and log it"""
        self.step_count += 1
        log_entry = {
            "step": self.step_count,
            "action": description,
            "tool": tool,
            "params": params,
            "timestamp": datetime.now().isoformat()
        }
        self.execution_log.append(log_entry)
        time.sleep(0.3)  # Simulate processing


# Demonstrate stateful agent behavior
agent = AgenticAISimulator()

# Single goal → Agent handles everything
result = agent.execute_goal(
    goal="Process loan application and recommend approval/denial",
    context={
        "customer_id": "CUST-8472",
        "customer_name": "Maria Lopez",
        "loan_amount": 50000,
        "monthly_income": 7083,
        "monthly_debt": 1200
    }
)

console.print("\n[bold green]Key Insight:[/bold green] Agent is STATEFUL and AUTONOMOUS!")
console.print("[dim]→ Agent remembers context across all steps[/dim]")
console.print("[dim]→ Agent plans and executes independently[/dim]")
console.print("[dim]→ Human sets goal, agent achieves it\n[/dim]")

---

## Demo 3: Side-by-Side Comparison — Same Task, Different Paradigms

**Task:** Analyze a customer dispute and draft resolution

**Compare:** GenAI (human-orchestrated) vs Agentic AI (self-orchestrated)

In [ ]:
# Banking scenario: Customer dispute
dispute_context = {
    "dispute_id": "DISP-4477",
    "customer_id": "CUST-9923",
    "customer_name": "John Smith",
    "transaction_id": "TXN-88234",
    "amount": 500.00,
    "merchant": "ElectronicsStore",
    "date": "2026-02-10",
    "dispute_reason": "Unauthorized charge"
}

console.print("[bold magenta]═══════════════════════════════════════════[/bold magenta]")
console.print("[bold magenta]  COMPARISON: GenAI vs Agentic AI         [/bold magenta]")
console.print("[bold magenta]  Task: Customer Dispute Resolution        [/bold magenta]")
console.print("[bold magenta]═══════════════════════════════════════════[/bold magenta]\n")

console.print(Panel(json.dumps(dispute_context, indent=2), title="📋 Dispute Details", border_style="cyan"))

# ============================================
# APPROACH 1: GenAI (Human-Orchestrated)
# ============================================
console.print("\n[bold yellow]━━━ APPROACH 1: GenAI (Reactive) ━━━[/bold yellow]\n")

start_time_genai = time.time()
genai_steps = []

# Step 1: Human queries dispute history
console.print("[cyan]👤 Human Action:[/cyan] Query database for customer's dispute history")
time.sleep(0.5)
dispute_history = {"total_disputes": 2, "resolved_in_favor": 2}  # Manual DB query
genai_steps.append("Human: Manual DB query")
console.print(f"[green]  → Found: {dispute_history}[/green]\n")

# Step 2: Human asks GenAI to draft response
console.print("[cyan]👤 Human Action:[/cyan] Prompt GenAI to draft customer response")
genai_prompt = f"""Draft email to customer about dispute {dispute_context['dispute_id']}:
- Customer: {dispute_context['customer_name']}
- Amount: ${dispute_context['amount']}
- Merchant: {dispute_context['merchant']}
- Customer has {dispute_history['total_disputes']} previous disputes, all resolved in their favor
- Tone: Professional, reassuring"""
console.print(f"[dim]  Prompt: {genai_prompt[:100]}...[/dim]")
time.sleep(0.5)
genai_steps.append("GenAI: Draft email")
email_draft = "Dear Mr. Smith, We've received your dispute regarding the $500 charge..."
console.print(f"[green]  → Generated: {email_draft[:50]}...[/green]\n")

# Step 3: Human reviews and sends
console.print("[cyan]👤 Human Action:[/cyan] Review email, approve, send via email system")
time.sleep(0.5)
genai_steps.append("Human: Review and send")
console.print("[green]  → Email sent[/green]\n")

genai_time = time.time() - start_time_genai

console.print(f"[bold]Total Steps:[/bold] {len(genai_steps)}")
console.print(f"[bold]Simulated Time:[/bold] {genai_time:.1f}s")
console.print(f"[bold]Human Interactions:[/bold] 3 (query, prompt, send)")
console.print(f"[bold]AI Interactions:[/bold] 1 (draft email only)\n")

# ============================================
# APPROACH 2: Agentic AI (Self-Orchestrated)
# ============================================
console.print("\n[bold yellow]━━━ APPROACH 2: Agentic AI (Proactive) ━━━[/bold yellow]\n")

start_time_agent = time.time()

class DisputeAgent:
    def __init__(self):
        self.memory = {}
        self.actions_taken = []
    
    def resolve_dispute(self, dispute_data: Dict):
        console.print("[cyan]🤖 Agent:[/cyan] Received goal - resolve dispute autonomously\n")
        self.memory.update(dispute_data)
        
        # Agent plans
        console.print("[green]📋 Agent Planning:[/green]")
        plan = [
            "1. Query customer dispute history",
            "2. Analyze pattern (genuine vs fraudulent)",
            "3. Draft customer response",
            "4. Queue for human review"
        ]
        for step in plan:
            console.print(f"  {step}")
        console.print()
        
        # Agent executes autonomously
        # Step 1: Query DB (tool use)
        console.print("[green]⚙️  Executing Step 1:[/green] Query dispute history (Tool: query_db)")
        time.sleep(0.3)
        history = {"total_disputes": 2, "resolved_in_favor": 2}
        self.memory['dispute_history'] = history
        self.actions_taken.append("query_db")
        console.print(f"  ✓ Retrieved: {history}\n")
        
        # Step 2: Analyze (reasoning)
        console.print("[green]⚙️  Executing Step 2:[/green] Analyze pattern")
        time.sleep(0.3)
        analysis = "Customer has 100% valid dispute history → High likelihood this is genuine"
        self.memory['analysis'] = analysis
        self.actions_taken.append("analyze")
        console.print(f"  ✓ Analysis: {analysis}\n")
        
        # Step 3: Draft response (content generation)
        console.print("[green]⚙️  Executing Step 3:[/green] Draft customer response")
        time.sleep(0.3)
        email = f"""Dear {self.memory['customer_name']},

We've received your dispute for ${self.memory['amount']} at {self.memory['merchant']}.
Based on your excellent account history, we're expediting this investigation.

We'll have a resolution within 2 business days.

Reference: {self.memory['dispute_id']}"""
        self.memory['email_draft'] = email
        self.actions_taken.append("draft_email")
        console.print(f"  ✓ Drafted: {email[:80]}...\n")
        
        # Step 4: Queue for review
        console.print("[green]⚙️  Executing Step 4:[/green] Queue for analyst review")
        time.sleep(0.3)
        self.actions_taken.append("queue_review")
        console.print("  ✓ Queued with priority: HIGH (based on customer history)\n")
        
        return {
            "status": "queued_for_review",
            "actions_taken": self.actions_taken,
            "recommendation": "Approve - customer has perfect dispute history",
            "email_ready": True
        }

agent = DisputeAgent()
result = agent.resolve_dispute(dispute_context)

agent_time = time.time() - start_time_agent

console.print(f"[bold]Total Actions:[/bold] {len(result['actions_taken'])}")
console.print(f"[bold]Simulated Time:[/bold] {agent_time:.1f}s")
console.print(f"[bold]Human Interactions:[/bold] 1 (review queued result only)")
console.print(f"[bold]AI Interactions:[/bold] 4 (query, analyze, draft, queue)\n")

# ============================================
# COMPARISON
# ============================================
console.print("\n[bold magenta]═══════════════════════════════════════════[/bold magenta]")
console.print("[bold magenta]           COMPARISON SUMMARY               [/bold magenta]")
console.print("[bold magenta]═══════════════════════════════════════════[/bold magenta]\n")

table = Table(title="GenAI vs Agentic AI")
table.add_column("Metric", style="cyan")
table.add_column("GenAI", style="yellow")
table.add_column("Agentic AI", style="green")

table.add_row("Who Orchestrates", "Human", "AI")
table.add_row("Human Steps Required", "3", "1 (review only)")
table.add_row("AI Autonomy", "Low (assists)", "High (executes)")
table.add_row("Database Queries", "Human manual", "Agent tool call")
table.add_row("Context Management", "Human maintains", "Agent memory")
table.add_row("Workflow Completion", "Partial (human finishes)", "End-to-end")

console.print(table)

console.print("\n[bold]Key Insight:[/bold]")
console.print("[green]✓ GenAI:[/green] Human drives workflow, AI assists with content")
console.print("[green]✓ Agentic:[/green] AI drives workflow, human validates result\n")

---

## Demo 4: The Autonomy Spectrum (5 Levels)

**Concept:** Banking AI exists on a spectrum from Level 0 (no AI) to Level 4 (autonomous)

**Example:** Fraud detection at different autonomy levels

In [ ]:
# Simulated suspicious transaction
transaction = {
    "id": "TXN-9981",
    "customer_id": "CUST-4429",
    "amount": 1200,
    "merchant": "ElectronicsStore",
    "location": "New York",
    "fraud_score": 65  # Medium risk
}

console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]")
console.print("[bold magenta]   THE AUTONOMY SPECTRUM: Fraud Detection      [/bold magenta]")
console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]\n")

console.print(Panel(json.dumps(transaction, indent=2), title="🚨 Suspicious Transaction", border_style="red"))

# ============================================
# LEVEL 0: No AI
# ============================================
console.print("\n[bold cyan]LEVEL 0: No AI (Manual)[/bold cyan]")
console.print("[dim]Human does everything manually\n[/dim]")

console.print("1. Analyst manually reviews transaction")
time.sleep(0.3)
console.print("2. Analyst checks customer history in system")
time.sleep(0.3)
console.print("3. Analyst compares against fraud patterns (memory/checklist)")
time.sleep(0.3)
console.print("4. Analyst decides: APPROVE")
time.sleep(0.3)
console.print("5. Analyst clicks button to approve\n")
console.print("[yellow]Time:[/yellow] ~10 minutes per transaction")
console.print("[yellow]Throughput:[/yellow] ~40 transactions/day per analyst\n")

# ============================================
# LEVEL 1: AI Suggestions
# ============================================
console.print("\n[bold cyan]LEVEL 1: AI Suggestions[/bold cyan]")
console.print("[dim]AI scores, human decides and executes\n[/dim]")

console.print("1. AI Model scores transaction: 65/100 (Medium Risk)")
time.sleep(0.3)
console.print("2. AI Suggests: 'REVIEW - Higher than usual amount, new merchant'")
time.sleep(0.3)
console.print("3. Analyst reviews AI reasoning + checks customer history")
time.sleep(0.3)
console.print("4. Analyst decides: APPROVE (customer recently moved)")
time.sleep(0.3)
console.print("5. Analyst clicks button to approve\n")
console.print("[yellow]Time:[/yellow] ~3 minutes per transaction")
console.print("[yellow]Throughput:[/yellow] ~150 transactions/day per analyst")
console.print("[green]Value:[/green] AI focuses analyst attention, but human still reviews all\n")

# ============================================
# LEVEL 2: AI Assistance
# ============================================
console.print("\n[bold cyan]LEVEL 2: AI Assistance[/bold cyan]")
console.print("[dim]AI executes WITH human approval each step\n[/dim]")

console.print("1. AI: 'Transaction scored 65/100. Shall I check customer history?'")
console.print("   Analyst: [APPROVE]")
time.sleep(0.3)
console.print("2. AI: [Queries DB] 'Customer has 3-year history, no fraud. Approve transaction?'")
console.print("   Analyst: [APPROVE]")
time.sleep(0.3)
console.print("3. AI: [Executes approval] 'Transaction approved. Log to audit trail?'")
console.print("   Analyst: [APPROVE]")
time.sleep(0.3)
console.print("4. AI: [Logs decision] 'Complete.'\n")
console.print("[yellow]Time:[/yellow] ~2 minutes per transaction")
console.print("[yellow]Throughput:[/yellow] ~200 transactions/day per analyst")
console.print("[green]Value:[/green] AI handles execution, human provides oversight at checkpoints\n")

# ============================================
# LEVEL 3: AI Automation
# ============================================
console.print("\n[bold cyan]LEVEL 3: AI Automation[/bold cyan]")
console.print("[dim]AI executes autonomously, human monitors exceptions\n[/dim]")

console.print("1. AI: [Scores] 65/100 (Medium Risk)")
time.sleep(0.2)
console.print("2. AI: [Checks threshold] 65 < 80 (auto-approve threshold)")
time.sleep(0.2)
console.print("3. AI: [Queries DB] Customer: clean 3-year history")
time.sleep(0.2)
console.print("4. AI: [Executes] Transaction APPROVED")
time.sleep(0.2)
console.print("5. AI: [Logs] Audit trail complete")
time.sleep(0.2)
console.print("6. AI: [Reports] Daily summary to analyst (no exceptions today)\n")
console.print("[yellow]Time:[/yellow] ~5 seconds per transaction")
console.print("[yellow]Throughput:[/yellow] ~10,000 transactions/day (automated)")
console.print("[yellow]Analyst Time:[/yellow] ~30 min/day reviewing exceptions only")
console.print("[green]Value:[/green] Handles 95%+ of cases end-to-end, analyst handles edge cases\n")

# ============================================
# LEVEL 4: AI Autonomy
# ============================================
console.print("\n[bold cyan]LEVEL 4: AI Autonomy (Self-Correcting)[/bold cyan]")
console.print("[dim]AI executes and self-corrects, human audits periodically\n[/dim]")

console.print("1. AI: [Scores] 65/100")
time.sleep(0.2)
console.print("2. AI: [Queries DB] → ERROR: Database timeout")
time.sleep(0.2)
console.print("3. AI: [Self-correction] Recognized transient error, retry in 30s")
time.sleep(0.2)
console.print("4. AI: [Retry] → SUCCESS: Customer clean history")
time.sleep(0.2)
console.print("5. AI: [Executes] Transaction APPROVED")
time.sleep(0.2)
console.print("6. AI: [Self-verification] Validates approval meets all policies")
time.sleep(0.2)
console.print("7. AI: [Logs] Includes retry attempt in audit trail")
time.sleep(0.2)
console.print("8. AI: Weekly report to analyst for audit\n")
console.print("[yellow]Time:[/yellow] ~5 seconds per transaction (handles own errors)")
console.print("[yellow]Throughput:[/yellow] ~10,000+ transactions/day")
console.print("[yellow]Analyst Time:[/yellow] ~2 hours/week auditing logs")
console.print("[green]Value:[/green] Fully autonomous with self-correction, minimal human oversight\n")

# ============================================
# COMPARISON
# ============================================
console.print("\n[bold magenta]═══════════════════════════════════════════════[/bold magenta]")
console.print("[bold magenta]           AUTONOMY LEVEL COMPARISON            [/bold magenta]")
console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]\n")

table = Table(title="Fraud Detection Across Autonomy Levels")
table.add_column("Level", style="cyan")
table.add_column("Human Effort", style="yellow")
table.add_column("Throughput", style="green")
table.add_column("Risk", style="red")

table.add_row("0 - Manual", "HIGH (10 min each)", "40/day", "Low (human validates)")
table.add_row("1 - Suggestions", "MEDIUM (3 min each)", "150/day", "Low (human decides)")
table.add_row("2 - Assistance", "MEDIUM (2 min each)", "200/day", "Low (human approves)")
table.add_row("3 - Automation", "LOW (exceptions only)", "10,000/day", "Medium (auto-execute)")
table.add_row("4 - Autonomy", "MINIMAL (periodic audit)", "10,000+/day", "Medium-High (self-correct)")

console.print(table)

console.print("\n[bold]Banking Principle:[/bold]")
console.print("[dim]→ Customer-facing: Level 1-2 (high control)[/dim]")
console.print("[dim]→ Back-office ops: Level 3-4 (efficiency)[/dim]")
console.print("[dim]→ Never skip levels - build trust gradually\n[/dim]")

---

## Demo 5: Agent Capabilities — Planning, Tools, Memory, Reflection

**Concept:** What makes Agentic AI different from GenAI

**Example:** Loan processing agent showcasing all 4 capabilities

In [ ]:
class LoanProcessingAgent:
    """Demonstrates 4 key agent capabilities"""
    
    def __init__(self):
        # CAPABILITY 1: MEMORY
        self.memory = {}
        self.conversation_history = []
        
        # Simulated tools available to agent
        self.available_tools = {
            "query_credit_bureau": self.tool_credit_bureau,
            "calculate_dti": self.tool_calculate_dti,
            "check_policy": self.tool_check_policy,
            "send_email": self.tool_send_email
        }
    
    def process_loan(self, application: Dict):
        """Process loan application using all 4 agent capabilities"""
        console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]")
        console.print("[bold magenta]    AGENT CAPABILITIES DEMONSTRATION            [/bold magenta]")
        console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]\n")
        
        console.print(Panel(json.dumps(application, indent=2), 
                           title="📄 Loan Application", border_style="cyan"))
        
        # Store initial context in MEMORY
        self.memory.update(application)
        
        # ============================================
        # CAPABILITY 1: PLANNING
        # ============================================
        console.print("\n[bold green]🧠 CAPABILITY 1: PLANNING[/bold green]")
        console.print("[dim]Agent decomposes goal into steps\n[/dim]")
        
        plan = [
            {"step": 1, "action": "Retrieve credit score", "tool": "query_credit_bureau"},
            {"step": 2, "action": "Calculate debt-to-income ratio", "tool": "calculate_dti"},
            {"step": 3, "action": "Check lending policy", "tool": "check_policy"},
            {"step": 4, "action": "Make recommendation", "tool": None},
            {"step": 5, "action": "Send notification email", "tool": "send_email"}
        ]
        
        for item in plan:
            tool_info = f" (Tool: {item['tool']})" if item['tool'] else ""
            console.print(f"  Step {item['step']}: {item['action']}{tool_info}")
        
        console.print("\n[green]✓ Plan created with dependencies identified[/green]\n")
        
        # ============================================
        # CAPABILITY 2: TOOL USE
        # ============================================
        console.print("\n[bold green]🔧 CAPABILITY 2: TOOL USE[/bold green]")
        console.print("[dim]Agent can call external systems\n[/dim]")
        
        # Execute Step 1: Credit Bureau API
        console.print("[cyan]Executing:[/cyan] Step 1 - Query credit bureau")
        credit_result = self.use_tool("query_credit_bureau", 
                                     {"customer_id": self.memory['customer_id']})
        self.memory['credit_score'] = credit_result['score']
        console.print(f"[green]✓ Tool result:[/green] Credit score = {credit_result['score']}\n")
        
        # Execute Step 2: Calculate DTI
        console.print("[cyan]Executing:[/cyan] Step 2 - Calculate DTI")
        dti_result = self.use_tool("calculate_dti", {
            "monthly_income": self.memory['monthly_income'],
            "monthly_debt": self.memory['monthly_debt']
        })
        self.memory['dti_ratio'] = dti_result['dti']
        console.print(f"[green]✓ Tool result:[/green] DTI = {dti_result['dti']}%\n")
        
        # ============================================
        # CAPABILITY 3: MEMORY
        # ============================================
        console.print("\n[bold green]💾 CAPABILITY 3: MEMORY[/bold green]")
        console.print("[dim]Agent remembers context across all steps\n[/dim]")
        
        console.print("[cyan]Agent's Current Memory State:[/cyan]")
        memory_snapshot = {
            "customer_name": self.memory['customer_name'],
            "loan_amount": self.memory['loan_amount'],
            "credit_score": self.memory.get('credit_score', 'Not yet retrieved'),
            "dti_ratio": self.memory.get('dti_ratio', 'Not yet calculated'),
            "steps_completed": 2
        }
        for key, value in memory_snapshot.items():
            console.print(f"  {key}: {value}")
        console.print("\n[green]✓ Agent can reference any previous data without re-asking[/green]\n")
        
        # Execute Step 3: Check Policy (uses memory!)
        console.print("[cyan]Executing:[/cyan] Step 3 - Check policy (uses remembered credit_score & dti)")
        policy_result = self.use_tool("check_policy", {
            "credit_score": self.memory['credit_score'],  # From memory!
            "dti_ratio": self.memory['dti_ratio']  # From memory!
        })
        self.memory['meets_policy'] = policy_result['approved']
        console.print(f"[green]✓ Tool result:[/green] {policy_result['message']}\n")
        
        # ============================================
        # CAPABILITY 4: REFLECTION
        # ============================================
        console.print("\n[bold green]🔍 CAPABILITY 4: REFLECTION[/bold green]")
        console.print("[dim]Agent self-evaluates and corrects\n[/dim]")
        
        # Generate recommendation
        console.print("[cyan]Agent Reasoning:[/cyan]")
        console.print(f"  Credit score: {self.memory['credit_score']} (min required: 680)")
        console.print(f"  DTI ratio: {self.memory['dti_ratio']}% (max allowed: 43%)")
        console.print(f"  Loan amount: ${self.memory['loan_amount']}")
        
        # Self-check: Does this make sense?
        console.print("\n[yellow]Agent Self-Check:[/yellow]")
        if self.memory['credit_score'] >= 680 and self.memory['dti_ratio'] <= 43:
            recommendation = "APPROVE"
            console.print("  ✓ Credit score meets minimum")
            console.print("  ✓ DTI within acceptable range")
            console.print("  ✓ All policy requirements satisfied")
        else:
            recommendation = "DENY"
            console.print("  ✗ Does not meet all policy requirements")
        
        self.memory['recommendation'] = recommendation
        
        # Verify recommendation matches policy check
        console.print("\n[yellow]Agent Verification:[/yellow]")
        if (recommendation == "APPROVE") == self.memory['meets_policy']:
            console.print("  ✓ Recommendation matches policy check - CONSISTENT")
        else:
            console.print("  ✗ INCONSISTENCY DETECTED - Would re-evaluate")
        
        console.print(f"\n[green]✓ Final Recommendation:[/green] {recommendation}\n")
        
        # Send notification
        console.print("[cyan]Executing:[/cyan] Step 5 - Send notification email")
        email_result = self.use_tool("send_email", {
            "to": self.memory['customer_email'],
            "subject": f"Loan Application Decision: {recommendation}",
            "body": f"Dear {self.memory['customer_name']}, your ${self.memory['loan_amount']} loan has been {recommendation.lower()}d."
        })
        console.print(f"[green]✓ Email sent to {self.memory['customer_email']}[/green]\n")
        
        # ============================================
        # FINAL RESULT
        # ============================================
        console.print("\n[bold magenta]═══════════════════════════════════════════════[/bold magenta]")
        console.print("[bold magenta]           AGENT COMPLETED AUTONOMOUSLY          [/bold magenta]")
        console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]\n")
        
        result = {
            "customer": self.memory['customer_name'],
            "recommendation": recommendation,
            "credit_score": self.memory['credit_score'],
            "dti_ratio": self.memory['dti_ratio'],
            "reasoning": f"Credit {self.memory['credit_score']}, DTI {self.memory['dti_ratio']}%",
            "tools_used": 4,
            "self_verified": True
        }
        
        console.print(Panel(json.dumps(result, indent=2), 
                           title="✅ Agent Result", border_style="green"))
        
        return result
    
    def use_tool(self, tool_name: str, params: Dict) -> Dict:
        """Execute a tool and return result"""
        console.print(f"[dim]  → Calling tool: {tool_name}({json.dumps(params)})[/dim]")
        time.sleep(0.3)
        return self.available_tools[tool_name](params)
    
    # Simulated tools
    def tool_credit_bureau(self, params: Dict) -> Dict:
        return {"score": 720, "source": "Experian"}
    
    def tool_calculate_dti(self, params: Dict) -> Dict:
        dti = (params['monthly_debt'] / params['monthly_income']) * 100
        return {"dti": round(dti, 1)}
    
    def tool_check_policy(self, params: Dict) -> Dict:
        approved = params['credit_score'] >= 680 and params['dti_ratio'] <= 43
        return {
            "approved": approved,
            "message": "Meets policy" if approved else "Does not meet policy"
        }
    
    def tool_send_email(self, params: Dict) -> Dict:
        return {"status": "sent", "message_id": "MSG-88291"}


# Run the demo
agent = LoanProcessingAgent()

loan_application = {
    "customer_id": "CUST-7621",
    "customer_name": "Sarah Johnson",
    "customer_email": "sarah.j@email.com",
    "loan_amount": 50000,
    "monthly_income": 7083,
    "monthly_debt": 1200
}

result = agent.process_loan(loan_application)

console.print("\n[bold]Key Insights:[/bold]")
console.print("[green]✓ PLANNING:[/green] Agent breaks down goal into steps")
console.print("[green]✓ TOOLS:[/green] Agent calls APIs/databases autonomously")
console.print("[green]✓ MEMORY:[/green] Agent maintains context across all steps")
console.print("[green]✓ REFLECTION:[/green] Agent self-verifies correctness\n")

---

## Demo 6: When GenAI Fails (Multi-Step Workflows)

**Concept:** GenAI limitations become obvious in complex banking workflows

**Example:** Monthly regulatory report generation

In [ ]:
console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]")
console.print("[bold magenta]   WHEN GENAI FAILS: Complex Workflow Example   [/bold magenta]")
console.print("[bold magenta]   Task: Monthly AML Regulatory Report          [/bold magenta]")
console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]\n")

# Required workflow steps
required_steps = [
    "1. Query transaction monitoring system (1.2M transactions)",
    "2. Query case management system (3,847 alerts)",
    "3. Join data from both systems on case_id",
    "4. Calculate statistics (alert rate, SAR rate, false positives)",
    "5. Compare vs previous month",
    "6. Generate narrative explaining trends",
    "7. Format per regulatory template (15 required sections)",
    "8. Generate data source audit trail",
    "9. Queue for compliance officer approval",
    "10. Submit to regulator portal"
]

console.print("[bold cyan]Required Workflow Steps:[/bold cyan]")
for step in required_steps:
    console.print(f"  {step}")

# ============================================
# GENAI APPROACH: Human-orchestrated
# ============================================
console.print("\n\n[bold yellow]━━━ GenAI Approach (Reactive) ━━━[/bold yellow]\n")

console.print("[cyan]Step 1:[/cyan] Human manually queries transaction system")
time.sleep(0.3)
console.print("  → Exports CSV (30 minutes)")
console.print("  [dim]GenAI cannot access database[/dim]\n")

console.print("[cyan]Step 2:[/cyan] Human manually queries case system")
time.sleep(0.3)
console.print("  → Exports CSV (15 minutes)")
console.print("  [dim]GenAI cannot access database[/dim]\n")

console.print("[cyan]Step 3:[/cyan] Human uses Excel to join data")
time.sleep(0.3)
console.print("  → VLOOKUP formulas (20 minutes)")
console.print("  [dim]GenAI cannot manipulate spreadsheets[/dim]\n")

console.print("[cyan]Step 4:[/cyan] Human calculates statistics")
time.sleep(0.3)
console.print("  → Manual formulas (15 minutes)")
console.print("  [dim]GenAI could help if human provides data[/dim]\n")

console.print("[cyan]Step 5:[/cyan] Human compares vs last month")
time.sleep(0.3)
console.print("  → Opens previous report, manual comparison (10 minutes)")
console.print("  [dim]GenAI has no memory of previous reports[/dim]\n")

console.print("[cyan]Step 6:[/cyan] [green]GenAI HELPS![/green] Human prompts GenAI")
time.sleep(0.3)
console.print("  Human: 'Generate narrative: alerts up 8%, SARs flat at 23...'")
console.print("  GenAI: [generates professional narrative]")
console.print("  → 2 minutes")
console.print("  [green]GenAI excels at content generation![/green]\n")

console.print("[cyan]Step 7:[/cyan] Human formats per template")
time.sleep(0.3)
console.print("  → Copy/paste into regulatory template (20 minutes)")
console.print("  [dim]GenAI could help if human provides template[/dim]\n")

console.print("[cyan]Step 8-10:[/cyan] Human handles approval and submission")
time.sleep(0.3)
console.print("  → Manual workflow (30 minutes)")
console.print("  [dim]GenAI cannot interact with systems[/dim]\n")

console.print("[bold red]GenAI Total Time:[/bold red] ~2.5 hours analyst time")
console.print("[bold red]GenAI Limitation:[/bold red] Helped with 1/10 steps (narrative only)\n")

# ============================================
# AGENTIC APPROACH: Self-orchestrated
# ============================================
console.print("\n\n[bold yellow]━━━ Agentic AI Approach (Proactive) ━━━[/bold yellow]\n")

console.print("[cyan]Human:[/cyan] 'Generate monthly AML report for January 2026'")
console.print("\n[green]Agent Autonomous Execution:[/green]\n")

steps_log = [
    ("Query transaction monitoring system", "API call", "1.2M records retrieved"),
    ("Query case management system", "API call", "3,847 alerts retrieved"),
    ("Join datasets on case_id", "Data processing", "Joined successfully"),
    ("Calculate required statistics", "Computation", "Alert rate: 0.32%, SAR rate: 0.6%"),
    ("Retrieve previous month report", "Database query", "December 2025 data loaded"),
    ("Compare trends", "Analysis", "Alerts +8%, SARs flat"),
    ("Generate narrative", "LLM generation", "Professional summary created"),
    ("Format per template", "Template engine", "15 sections populated"),
    ("Generate audit trail", "Logging", "Data sources documented"),
    ("Queue for approval", "Workflow API", "Sent to compliance officer")
]

for i, (action, method, result) in enumerate(steps_log, 1):
    console.print(f"[cyan]Step {i}:[/cyan] {action}")
    time.sleep(0.2)
    console.print(f"  Method: {method}")
    console.print(f"  Result: {result}")
    console.print()

console.print("[bold green]Agent Total Time:[/bold green] ~10 minutes (automated)")
console.print("[bold green]Human Time:[/bold green] ~15 minutes (review & approve)")
console.print("[bold green]Agent Coverage:[/bold green] Handled 10/10 steps end-to-end\n")

# ============================================
# COMPARISON
# ============================================
console.print("\n[bold magenta]═══════════════════════════════════════════════[/bold magenta]")
console.print("[bold magenta]              COMPARISON SUMMARY                 [/bold magenta]")
console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]\n")

table = Table(title="Complex Workflow: GenAI vs Agentic AI")
table.add_column("Metric", style="cyan")
table.add_column("GenAI", style="yellow")
table.add_column("Agentic AI", style="green")

table.add_row("Human Time", "~2.5 hours", "~15 minutes")
table.add_row("Steps Automated", "1/10 (narrative only)", "10/10 (end-to-end)")
table.add_row("Database Access", "❌ Human manual", "✅ Agent tool calls")
table.add_row("Context Management", "❌ Human maintains", "✅ Agent memory")
table.add_row("Multi-step Planning", "❌ Human orchestrates", "✅ Agent plans")
table.add_row("System Integration", "❌ Manual export/import", "✅ API automation")
table.add_row("Time Savings", "Baseline", "90% reduction")

console.print(table)

console.print("\n[bold]Key Insight:[/bold]")
console.print("[red]✗ GenAI:[/red] Excels at SINGLE STEP content tasks")
console.print("[green]✓ Agentic:[/green] Required for MULTI-STEP workflows with tool use\n")

---

## Demo 7: Cost Analysis — Token Usage Comparison

**Concept:** GenAI and Agentic AI have different cost models

**Example:** Calculate real costs for banking use cases

In [ ]:
import tiktoken

# Claude Sonnet 4.5 pricing (Feb 2026)
PRICING = {
    "input_per_million": 3.00,
    "output_per_million": 15.00
}

encoder = tiktoken.get_encoding("cl100k_base")

def calculate_cost(input_text: str, output_text: str) -> Dict:
    """Calculate API cost for a single call"""
    input_tokens = len(encoder.encode(input_text))
    output_tokens = len(encoder.encode(output_text))
    
    input_cost = (input_tokens / 1_000_000) * PRICING["input_per_million"]
    output_cost = (output_tokens / 1_000_000) * PRICING["output_per_million"]
    total_cost = input_cost + output_cost
    
    return {
        "input_tokens": input_tokens,
        "output_tokens": output_tokens,
        "total_tokens": input_tokens + output_tokens,
        "cost_usd": total_cost
    }

console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]")
console.print("[bold magenta]       COST ANALYSIS: GenAI vs Agentic AI       [/bold magenta]")
console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]\n")

# ============================================
# USE CASE 1: Customer Email Response
# ============================================
console.print("[bold cyan]Use Case 1: Customer Service Email[/bold cyan]\n")

email_input = """Draft professional response to customer complaint:
Customer: John Smith
Issue: Charged $35 overdraft fee
Account went negative by $5.30
Requesting fee waiver
History: 3rd overdraft in 6 months
Policy: Cannot waive (not first offense)
Suggest: Overdraft protection service"""

email_output = """Dear Mr. Smith,

Thank you for contacting us regarding the $35 overdraft fee. I understand your concern.

Our policy allows fee waivers for first-time overdrafts within 12 months. Your account shows this is your third overdraft in the past six months, which means we're unable to waive the fee on this occasion.

To help prevent future fees, I'd like to tell you about our Overdraft Protection service, which transfers from savings to cover shortfalls for just $5 instead of $35.

Would you like to enroll?

Best regards,
[Agent Name]"""

genai_email_cost = calculate_cost(email_input, email_output)

console.print("[yellow]GenAI Cost (single call):[/yellow]")
console.print(f"  Input tokens: {genai_email_cost['input_tokens']}")
console.print(f"  Output tokens: {genai_email_cost['output_tokens']}")
console.print(f"  Cost per email: ${genai_email_cost['cost_usd']:.4f}")
console.print(f"  Cost for 1000 emails/month: ${genai_email_cost['cost_usd'] * 1000:.2f}\n")

# Agentic would need multiple calls (query customer history, check policy, draft email)
agentic_email_calls = [
    calculate_cost("Query customer overdraft history for CUST-123", "3 overdrafts in past 6 months"),
    calculate_cost("Check overdraft fee waiver policy", "First offense only within 12 months"),
    calculate_cost(email_input, email_output)
]
agentic_email_total = sum(c['cost_usd'] for c in agentic_email_calls)

console.print("[green]Agentic AI Cost (3 tool calls + generation):[/green]")
console.print(f"  Total calls: {len(agentic_email_calls)}")
console.print(f"  Cost per email: ${agentic_email_total:.4f}")
console.print(f"  Cost for 1000 emails/month: ${agentic_email_total * 1000:.2f}\n")

console.print(f"[bold]Comparison:[/bold] Agentic is {agentic_email_total / genai_email_cost['cost_usd']:.1f}x more expensive")
console.print(f"[dim]But: Agentic is fully automated (no human DB queries)[/dim]\n")

# ============================================
# USE CASE 2: Loan Application Processing
# ============================================
console.print("\n[bold cyan]Use Case 2: Loan Application Processing[/bold cyan]\n")

# GenAI: 4 separate prompts (human-orchestrated)
genai_loan_calls = [
    calculate_cost("Summarize this 5-page loan application: [5000 words]", "[500 word summary]"),
    calculate_cost("Calculate DTI: income $85K, debt $1200/month", "DTI = 16.9%"),
    calculate_cost("Does credit score 720, DTI 16.9% meet policy?", "Yes, meets lending guidelines"),
    calculate_cost("Draft approval email for $50K loan", "[200 word email]")
]
genai_loan_total = sum(c['cost_usd'] for c in genai_loan_calls)

console.print("[yellow]GenAI Cost (4 separate human-prompted calls):[/yellow]")
console.print(f"  Total calls: {len(genai_loan_calls)}")
console.print(f"  Cost per loan: ${genai_loan_total:.4f}")
console.print(f"  Cost for 100 loans/month: ${genai_loan_total * 100:.2f}")
console.print(f"  Human time: ~30 min/loan\n")

# Agentic: Single goal, 8 tool calls
agentic_loan_calls = [
    calculate_cost("Process loan application LOAN-8834", "Planning: 5 steps identified"),
    calculate_cost("[Tool] Query credit bureau", "Score: 720"),
    calculate_cost("[Tool] Query income verification", "Verified: $85K"),
    calculate_cost("[Tool] Calculate DTI", "DTI: 16.9%"),
    calculate_cost("[Tool] Check lending policy", "Approved: meets all requirements"),
    calculate_cost("Generate recommendation", "APPROVE: credit 720, DTI 16.9%"),
    calculate_cost("[Tool] Generate approval email", "[200 word email]"),
    calculate_cost("[Tool] Update loan status in DB", "Status: APPROVED")
]
agentic_loan_total = sum(c['cost_usd'] for c in agentic_loan_calls)

console.print("[green]Agentic AI Cost (8 autonomous tool calls):[/green]")
console.print(f"  Total calls: {len(agentic_loan_calls)}")
console.print(f"  Cost per loan: ${agentic_loan_total:.4f}")
console.print(f"  Cost for 100 loans/month: ${agentic_loan_total * 100:.2f}")
console.print(f"  Human time: ~5 min/loan (review only)\n")

console.print(f"[bold]API Cost:[/bold] Agentic is {agentic_loan_total / genai_loan_total:.1f}x more expensive")
console.print(f"[bold]Labor Savings:[/bold] 25 min/loan × 100 loans = 2500 min = 42 hours saved/month")
console.print(f"[dim]At $50/hour analyst cost: ${50 * 42:.0f}/month saved[/dim]")
console.print(f"[dim]ROI: ${50 * 42:.0f} saved vs ${agentic_loan_total * 100 - genai_loan_total * 100:.2f} extra API cost[/dim]\n")

# ============================================
# SUMMARY TABLE
# ============================================
console.print("\n[bold magenta]═══════════════════════════════════════════════[/bold magenta]")
console.print("[bold magenta]              COST SUMMARY                      [/bold magenta]")
console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]\n")

table = Table(title="Cost Comparison")
table.add_column("Use Case", style="cyan")
table.add_column("GenAI Cost", style="yellow")
table.add_column("Agentic Cost", style="green")
table.add_column("Labor Savings", style="magenta")

table.add_row(
    "Customer Email (1000/mo)",
    f"${genai_email_cost['cost_usd'] * 1000:.2f}",
    f"${agentic_email_total * 1000:.2f}",
    "Minimal (still needs review)"
)

table.add_row(
    "Loan Processing (100/mo)",
    f"${genai_loan_total * 100:.2f}",
    f"${agentic_loan_total * 100:.2f}",
    "42 hours ($2,100)"
)

console.print(table)

console.print("\n[bold]Cost Insights:[/bold]")
console.print("[yellow]→ GenAI:[/yellow] Lower API cost, higher human cost (orchestration)")
console.print("[green]→ Agentic:[/green] Higher API cost, lower human cost (automation)")
console.print("[cyan]→ Decision:[/cyan] Simple tasks = GenAI. Complex workflows = Agentic\n")

---

## Demo 8: Guardrails for Agentic Systems

**Concept:** Agentic AI needs safety controls

**Example:** Implement common guardrails for banking agents

In [ ]:
class GuardrailedAgent:
    """Agent with banking-appropriate safety controls"""
    
    def __init__(self):
        # GUARDRAIL 1: Transaction Limits
        self.limits = {
            "max_transaction_amount": 10000,
            "max_daily_volume": 50000,
            "max_api_calls_per_task": 20,
            "max_execution_time_seconds": 300
        }
        
        # GUARDRAIL 2: Confidence Thresholds
        self.confidence_thresholds = {
            "auto_approve": 0.95,  # >95% confidence
            "escalate": 0.70        # <70% escalate to human
        }
        
        # GUARDRAIL 3: Audit Logging
        self.audit_log = []
        
        # GUARDRAIL 4: Allowed Actions
        self.allowed_actions = [
            "query_database",
            "send_email",
            "update_status",
            "generate_report"
        ]
        self.forbidden_actions = [
            "delete_records",
            "modify_balances",
            "grant_access"
        ]
        
        # Track state
        self.daily_volume = 0
        self.api_calls_this_task = 0
        self.task_start_time = None
    
    def process_transaction(self, transaction: Dict) -> Dict:
        """Process transaction with guardrails"""
        console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]")
        console.print("[bold magenta]       GUARDRAILED AGENT DEMONSTRATION          [/bold magenta]")
        console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]\n")
        
        console.print(Panel(json.dumps(transaction, indent=2), 
                           title="💳 Transaction Request", border_style="cyan"))
        
        self.task_start_time = time.time()
        self.api_calls_this_task = 0
        
        # ============================================
        # GUARDRAIL 1: Amount Limit Check
        # ============================================
        console.print("\n[bold yellow]🛡️  GUARDRAIL 1: Transaction Amount Limit[/bold yellow]")
        console.print(f"  Transaction amount: ${transaction['amount']}")
        console.print(f"  Maximum allowed: ${self.limits['max_transaction_amount']}")
        
        if transaction['amount'] > self.limits['max_transaction_amount']:
            console.print("  [red]❌ BLOCKED: Exceeds agent authority[/red]")
            return self._escalate(
                transaction,
                f"Amount ${transaction['amount']} exceeds limit ${self.limits['max_transaction_amount']}"
            )
        console.print("  [green]✓ PASS: Within limits[/green]\n")
        
        # ============================================
        # GUARDRAIL 2: Daily Volume Check
        # ============================================
        console.print("[bold yellow]🛡️  GUARDRAIL 2: Daily Volume Limit[/bold yellow]")
        console.print(f"  Today's volume: ${self.daily_volume}")
        console.print(f"  This transaction: ${transaction['amount']}")
        console.print(f"  Daily limit: ${self.limits['max_daily_volume']}")
        
        if self.daily_volume + transaction['amount'] > self.limits['max_daily_volume']:
            console.print("  [red]❌ BLOCKED: Would exceed daily limit[/red]")
            return self._escalate(
                transaction,
                f"Daily volume limit reached: ${self.daily_volume} + ${transaction['amount']}"
            )
        console.print("  [green]✓ PASS: Within daily limit[/green]\n")
        
        # ============================================
        # GUARDRAIL 3: Confidence Check
        # ============================================
        console.print("[bold yellow]🛡️  GUARDRAIL 3: Confidence Threshold[/bold yellow]")
        
        # Simulate fraud detection confidence
        confidence = transaction.get('fraud_confidence', 0.85)
        console.print(f"  Fraud detection confidence: {confidence:.0%}")
        console.print(f"  Auto-approve threshold: {self.confidence_thresholds['auto_approve']:.0%}")
        console.print(f"  Escalation threshold: {self.confidence_thresholds['escalate']:.0%}")
        
        if confidence < self.confidence_thresholds['escalate']:
            console.print("  [red]❌ BLOCKED: Confidence too low[/red]")
            return self._escalate(
                transaction,
                f"Confidence {confidence:.0%} below threshold {self.confidence_thresholds['escalate']:.0%}"
            )
        elif confidence < self.confidence_thresholds['auto_approve']:
            console.print("  [yellow]⚠️  WARNING: Medium confidence - logging for review[/yellow]\n")
        else:
            console.print("  [green]✓ PASS: High confidence[/green]\n")
        
        # ============================================
        # GUARDRAIL 4: Action Allowlist
        # ============================================
        console.print("[bold yellow]🛡️  GUARDRAIL 4: Action Allowlist[/bold yellow]")
        
        requested_action = "approve_transaction"
        console.print(f"  Requested action: {requested_action}")
        console.print(f"  Allowed actions: {', '.join(self.allowed_actions)}")
        
        # Simulate attempting forbidden action
        if requested_action in self.forbidden_actions:
            console.print("  [red]❌ BLOCKED: Forbidden action[/red]")
            return self._escalate(
                transaction,
                f"Action '{requested_action}' not in allowlist"
            )
        console.print("  [green]✓ PASS: Action permitted[/green]\n")
        
        # ============================================
        # GUARDRAIL 5: Execution Time Limit
        # ============================================
        console.print("[bold yellow]🛡️  GUARDRAIL 5: Execution Time Limit[/bold yellow]")
        
        elapsed = time.time() - self.task_start_time
        console.print(f"  Elapsed time: {elapsed:.1f}s")
        console.print(f"  Maximum allowed: {self.limits['max_execution_time_seconds']}s")
        
        if elapsed > self.limits['max_execution_time_seconds']:
            console.print("  [red]❌ TIMEOUT: Exceeded execution limit[/red]")
            return self._escalate(
                transaction,
                f"Task timeout: {elapsed:.1f}s > {self.limits['max_execution_time_seconds']}s"
            )
        console.print("  [green]✓ PASS: Within time limit[/green]\n")
        
        # ============================================
        # GUARDRAIL 6: Comprehensive Audit Log
        # ============================================
        console.print("[bold yellow]🛡️  GUARDRAIL 6: Audit Logging[/bold yellow]")
        
        audit_entry = {
            "timestamp": datetime.now().isoformat(),
            "transaction_id": transaction['id'],
            "amount": transaction['amount'],
            "decision": "APPROVED",
            "confidence": confidence,
            "guardrails_passed": [
                "amount_limit",
                "daily_volume",
                "confidence_threshold",
                "action_allowlist",
                "execution_time"
            ],
            "api_calls_used": self.api_calls_this_task,
            "execution_time_seconds": elapsed
        }
        self.audit_log.append(audit_entry)
        
        console.print("  [green]✓ Logged to audit trail[/green]")
        console.print(f"  [dim]Audit entry #{len(self.audit_log)} created[/dim]\n")
        
        # ============================================
        # SUCCESS: All guardrails passed
        # ============================================
        self.daily_volume += transaction['amount']
        
        console.print("[bold green]✅ ALL GUARDRAILS PASSED - TRANSACTION APPROVED[/bold green]\n")
        
        result = {
            "status": "approved",
            "transaction_id": transaction['id'],
            "amount": transaction['amount'],
            "confidence": confidence,
            "guardrails_status": "all_passed",
            "audit_log_id": len(self.audit_log)
        }
        
        console.print(Panel(json.dumps(result, indent=2), 
                           title="✅ Agent Result", border_style="green"))
        
        return result
    
    def _escalate(self, transaction: Dict, reason: str) -> Dict:
        """Escalate to human when guardrails fail"""
        console.print("\n[bold red]🚨 ESCALATING TO HUMAN ANALYST[/bold red]")
        console.print(f"[red]Reason: {reason}[/red]\n")
        
        # Log escalation
        audit_entry = {
            "timestamp": datetime.now().isoformat(),
            "transaction_id": transaction['id'],
            "decision": "ESCALATED",
            "reason": reason
        }
        self.audit_log.append(audit_entry)
        
        return {
            "status": "escalated",
            "transaction_id": transaction['id'],
            "reason": reason,
            "requires_human_review": True
        }


# ============================================
# TEST SCENARIOS
# ============================================
agent = GuardrailedAgent()

# Scenario 1: Normal transaction (passes all guardrails)
console.print("[bold cyan]\n\nSCENARIO 1: Normal Transaction[/bold cyan]\n")
result1 = agent.process_transaction({
    "id": "TXN-001",
    "amount": 5000,
    "fraud_confidence": 0.97
})

# Scenario 2: Amount too high (fails guardrail)
console.print("\n\n[bold cyan]SCENARIO 2: Amount Exceeds Limit[/bold cyan]\n")
result2 = agent.process_transaction({
    "id": "TXN-002",
    "amount": 25000,  # Exceeds $10K limit
    "fraud_confidence": 0.98
})

# Scenario 3: Low confidence (fails guardrail)
console.print("\n\n[bold cyan]SCENARIO 3: Low Confidence Score[/bold cyan]\n")
result3 = agent.process_transaction({
    "id": "TXN-003",
    "amount": 3000,
    "fraud_confidence": 0.65  # Below 70% threshold
})

# ============================================
# SUMMARY
# ============================================
console.print("\n\n[bold magenta]═══════════════════════════════════════════════[/bold magenta]")
console.print("[bold magenta]          GUARDRAILS SUMMARY                    [/bold magenta]")
console.print("[bold magenta]═══════════════════════════════════════════════[/bold magenta]\n")

table = Table(title="Banking Agent Guardrails")
table.add_column("Guardrail", style="cyan")
table.add_column("Purpose", style="yellow")
table.add_column("Example Limit", style="green")

table.add_row("Transaction Limits", "Cap agent authority", "$10K per transaction")
table.add_row("Daily Volume", "Prevent runaway costs", "$50K per day")
table.add_row("Confidence Threshold", "Escalate uncertain cases", "<70% → human review")
table.add_row("Action Allowlist", "Prevent destructive actions", "No deletes, no balance mods")
table.add_row("Execution Time", "Prevent infinite loops", "5 minute timeout")
table.add_row("Audit Logging", "Regulatory compliance", "Log every decision")

console.print(table)

console.print("\n[bold]Key Principle:[/bold]")
console.print("[green]✓ Start with strict guardrails, loosen gradually based on observed behavior[/green]")
console.print("[green]✓ Every autonomous action must be logged for audit[/green]")
console.print("[green]✓ When in doubt, escalate to human\n[/green]")

---

## Summary & Key Takeaways

### What We Demonstrated

1. **GenAI is Stateless** — No memory between calls, human orchestrates
2. **Agentic AI is Stateful** — Maintains context, self-orchestrates
3. **Autonomy Spectrum** — 5 levels from manual to autonomous
4. **Agent Capabilities** — Planning, Tools, Memory, Reflection
5. **GenAI Limitations** — Breaks down in multi-step workflows
6. **Cost Tradeoffs** — Higher API cost vs lower human cost
7. **Guardrails** — Critical for safe banking deployment

### Decision Framework

**Use GenAI when:**
- Single-step task (draft email, summarize document)
- Content generation/transformation
- Human will review and execute
- Low cost priority

**Use Agentic AI when:**
- Multi-step workflow (reconciliation, report generation)
- Need tool access (databases, APIs)
- High-volume repetitive tasks
- Efficiency priority (even at higher API cost)

### Banking Best Practices

1. Start conservative (Level 1-2), increase autonomy gradually
2. Implement strict guardrails (limits, thresholds, allowlists)
3. Comprehensive audit logging (every decision tracked)
4. Human-in-loop for customer-facing decisions
5. Higher autonomy for back-office operations

---

## Next: Session 3 — RAG (Retrieval Augmented Generation)

We'll explore how to give AI access to banking knowledge bases, policy documents, and customer data.
